# Optimisation of a Neocortical Layer 5 Pyramidal Cell

This notebook shows you how to optimise the maximal conductance of Neocortical Layer 5 Pyramidal Cell as used in Markram et al. 2015.

Author of this script: Werner Van Geit @ Blue Brain Project

Choice of parameters, protocols and other settings was done by Etay Hay @ HUJI

What's described here is a more advanced use of BluePyOpt. We suggest to first go through the introductary example here: https://github.com/BlueBrain/BluePyOpt/blob/master/examples/simplecell/simplecell.ipynb

**If you use the methods in this notebook, we ask you to cite the following publications when publishing your research:**

Van Geit, W., M. Gevaert, G. Chindemi, C. Rössert, J.-D. Courcol, E. Muller, F. Schürmann, I. Segev, and H. Markram (2016, March). BluePyOpt: Leveraging open source software and cloud infrastructure to optimise model parameters in neuroscience. ArXiv e-prints.
http://arxiv.org/abs/1603.00500

Markram, H., E. Muller, S. Ramaswamy, M. W. Reimann, M. Abdellah, C. A. Sanchez, A. Ailamaki, L. Alonso-Nanclares, N. Antille, S. Arsever, et al. (2015). Reconstruction and simulation of neocortical microcircuitry. Cell 163(2), 456–492.
http://www.cell.com/abstract/S0092-8674%2815%2901191-5

Some of the modules loaded in this script are located in the L5PC example folder: https://github.com/BlueBrain/BluePyOpt/tree/master/examples/l5pc 

We first load the bluepyopt python module, the ephys submodule and some helper functionality

In [1]:
%load_ext autoreload
%autoreload

from __future__ import print_function

# directory with all the neuron models (*.mod)
# command to compile the mechanisms as we saw in the previous exercise session
!nrnivmodl mechanisms
import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import pprint
pp = pprint.PrettyPrinter(indent=2)

%matplotlib notebook
import matplotlib.pyplot as plt

# install & upgrade neurom
!pip install neurom --upgrade

/home/hbpschool2016/Documents/cell_optimization
mechanisms/IhDA.mod mechanisms/ampasyn.mod mechanisms/cabalan.mod mechanisms/cachan.mod mechanisms/capump.mod mechanisms/dop.mod mechanisms/hh3.mod mechanisms/kca.mod mechanisms/leak.mod mechanisms/nabalan.mod mechanisms/nmdasyn.mod mechanisms/pump.mod
IhDA.mod ampasyn.mod cabalan.mod cachan.mod capump.mod dop.mod hh3.mod kca.mod leak.mod nabalan.mod nmdasyn.mod pump.mod
"/home/hbpschool2016/local/nrn/share/nrn/libtool"  --mode=compile gcc -DHAVE_CONFIG_H  -I. -I.. -I"/home/hbpschool2016/local/nrn/include/nrn" -I"/home/hbpschool2016/local/nrn/x86_64/lib"      -g -O2 -c -o mod_func.lo mod_func.c
libtool: compile:  gcc -DHAVE_CONFIG_H -I. -I.. -I/home/hbpschool2016/local/nrn/include/nrn -I/home/hbpschool2016/local/nrn/x86_64/lib -g -O2 -c mod_func.c  -fPIC -DPIC -o .libs/mod_func.o
"/home/hbpschool2016/local/nrn/share/nrn/libtool"  --mode=link gcc -module  -g -O2    -o libnrnmech.la -rpath "/home/hbpschool2016/local/nrn/x86_64/lib"  IhDA.lo

Main Directories:
    - Morphology: The morphology file is stored here
    - Mechanisms: All model files (*.mod) for the cell are stored here
    - Config:     Contains the configuration files for the optimisation

Enable the code below to enable debug level logging

## Model description

### Morphology

We're using a complex reconstructed morphology of an L5PC cell. Let's visualise this with the BlueBrain NeuroM software:

In [2]:
# TODO: your group's morphology must be added to the directory morphology
morphology_filename = 'morphology/OH141125_A0_idA.ASC'

In [3]:
import neurom.viewer
neurom.viewer.draw(neurom.load_neuron(morphology_filename));

/home/hbpschool2016/local/pythonenv/local/lib/python2.7/site-packages/neurom/io/neurolucida.py:263: UserWarning: This is an experimental reader. There are no guarantees regarding ability to parse Neurolucida .asc files or correctness of output.
  warnings.warn(msg)
No handlers could be found for logger "neurom.io.neurolucida"


<IPython.core.display.Javascript object>

To load the morphology we create a NrnFileMorphology object. We set 'do_replace_axon' to True to replace the axon with a Axon Initial Segment.

In [4]:
morphology = ephys.morphologies.NrnFileMorphology(morphology_filename, do_replace_axon=True)

print(str(morphology))

morphology/OH141125_A0_idA.ASC


### Parameters

Since we have many parameters in this model, they are stored in a json file: https://github.com/BlueBrain/BluePyOpt/blob/master/examples/l5pc/config/parameters.json

In [5]:
# Example of json entry: Frozen Parameter for the equilibrum membrane potential
{
    "param_name": "e_pas",
    "sectionlist": "all",
    "type": "section",
    "dist_type": "uniform",
    "value": -75
},
# Example of json entry: Bounded Parameter
{
    "param_name": "gNaTs2_tbar_NaTs2_t",
    "mech": "NaTs2_t",
    "bounds": [
        0,
        1
    ],
    "dist_type": "uniform",
    "mech_param": "gNaTs2_tbar",
    "type": "range",
    "sectionlist": "somatic"
},
#Can see the file features in config 

({'bounds': [0, 1],
  'dist_type': 'uniform',
  'mech': 'NaTs2_t',
  'mech_param': 'gNaTs2_tbar',
  'param_name': 'gNaTs2_tbar_NaTs2_t',
  'sectionlist': 'somatic',
  'type': 'range'},)

In [6]:
import json
param_configs = json.load(open('config/parameters.json'))

In [7]:
print('{0:<25}: {1:<10}: {2:<15}: {3:<10}: {4:<10}'.format('Parameter Name',
                                                           'Par Type',
                                                           'Value',
                                                           'SecList',
                                                           'Distribution Type'))
print('-'*100)
for param_config in param_configs:
    
    string = '{0:<25}: {1:<10}: '.format(param_config['param_name'],
                                         param_config['type'])

    if 'value' in param_config:
        string += '{0:<15}: '.format(param_config['value'])
    elif 'range' in param_config['type']:
        string += '{0:<15}: '.format(param_config['bounds'])

    if 'sectionlist' in param_config:
        string += '{0:<10}: '.format(param_config['sectionlist'])
    else:
        string += '{0:<10}: '.format('NA')

    if 'dist_type' in param_config:
        
        string += '{0:<10}'.format(param_config['dist_type'])
    else:
        string += '{0:<10}'.format('NA')

    print(string)

Parameter Name           : Par Type  : Value          : SecList   : Distribution Type
----------------------------------------------------------------------------------------------------
g_pas                    : section   : 3e-05          : all       : uniform   
e_pas                    : section   : -75            : all       : uniform   
cm                       : section   : 1              : all       : uniform   
Ra                       : section   : 100            : all       : uniform   
v_init                   : global    : -65            : NA        : NA        
celsius                  : global    : 34             : NA        : NA        
ena                      : section   : 50             : somatic   : uniform   
ek                       : section   : -85            : somatic   : uniform   
gNaTs2_tbar_NaTs2_t      : range     : [0, 1]         : somatic   : uniform   
gSKv3_1bar_SKv3_1        : range     : [0, 1]         : somatic   : uniform   
gSK_E2bar_SK_E2        

The directory that contains this notebook has a module that will load all the parameters in BluePyOpt Parameter objects

In [8]:
import l5pc_model
parameters = l5pc_model.define_parameters()

for p in parameters:
    print('{0:<50}: {1:<10}'.format(p.name, p.value if p.frozen else p.bounds))

g_pas.all                                         : 3e-05     
e_pas.all                                         : -75       
cm.all                                            : 1         
Ra.all                                            : 100       
v_init                                            : -65       
celsius                                           : 34        
ena.somatic                                       : 50        
ek.somatic                                        : -85       
gNaTs2_tbar_NaTs2_t.somatic                       : [0, 1]    
gSKv3_1bar_SKv3_1.somatic                         : [0, 1]    
gSK_E2bar_SK_E2.somatic                           : [0, 0.1]  
gCa_HVAbar_Ca_HVA.somatic                         : [0, 0.001]
gCa_LVAstbar_Ca_LVAst.somatic                     : [0, 0.01] 
gamma_CaDynamics_E2.somatic                       : [0.0005, 0.05]
decay_CaDynamics_E2.somatic                       : [20, 1000]
gIhbar_Ih.somatic                                 :

As you can see there are two types of parameters, <b>parameters with a fixed value</b> and <b>parameters with bounds</b>. The latter will be optimised by the algorithm.

### Mechanism

We also need to add all the necessary mechanisms, like ion channels to the model. 
The configuration of the mechanisms is also stored in a json file, and can be loaded in a similar way.

In [22]:
mechanisms = l5pc_model.define_mechanisms() 

print('\n'.join(map(str, mechanisms)))

pas.all: pas at ['all']
leak.somatic: leak at ['somatic']
cabalan.somatic: cabalan at ['somatic']
cachan.somatic: cachan at ['somatic']
capump.somatic: capump at ['somatic']
hh3.somatic: hh3 at ['somatic']
kca.somatic: kca at ['somatic']
hd.somatic: hd at ['somatic']


# Cell model

With the morphology, mechanisms and parameters we can build the cell model

In [23]:
l5pc_cell = ephys.models.CellModel('l5pc', morph=morphology, mechs=mechanisms, params=parameters)
print(l5pc_cell)

l5pc:
  morphology:
    morphology/OH141125_A0_idA.ASC
  mechanisms:
    pas.all: pas at ['all']
    leak.somatic: leak at ['somatic']
    cabalan.somatic: cabalan at ['somatic']
    cachan.somatic: cachan at ['somatic']
    capump.somatic: capump at ['somatic']
    hh3.somatic: hh3 at ['somatic']
    kca.somatic: kca at ['somatic']
    hd.somatic: hd at ['somatic']
  params:
    g_pas.all: ['all'] g_pas = 3e-05
    e_pas.all: ['all'] e_pas = -75
    cm.all: ['all'] cm = 1
    Ra.all: ['all'] Ra = 100
    v_init: v_init = -65
    celsius: celsius = 34
    ena.somatic: ['somatic'] ena = 50
    ek.somatic: ['somatic'] ek = -85
    gNaTs2_tbar_NaTs2_t.somatic: ['somatic'] gNaTs2_tbar_NaTs2_t = [0, 1]
    gSKv3_1bar_SKv3_1.somatic: ['somatic'] gSKv3_1bar_SKv3_1 = [0, 1]
    gSK_E2bar_SK_E2.somatic: ['somatic'] gSK_E2bar_SK_E2 = [0, 0.1]
    gCa_HVAbar_Ca_HVA.somatic: ['somatic'] gCa_HVAbar_Ca_HVA = [0, 0.001]
    gCa_LVAstbar_Ca_LVAst.somatic: ['somatic'] gCa_LVAstbar_Ca_LVAst = [0, 0.01]


For use in the cell evaluator later, we need to make a list of the name of the parameters we are going to optimise.
These are the parameters that are not frozen.

In [24]:
param_names = [param.name for param in l5pc_cell.params.values() if not param.frozen]   

## Protocols

Now that we have a cell model, we can apply protocols to it. The protocols are also stored in a json file.

In [25]:
proto_configs = json.load(open('config/protocols.json'))

for key, value in proto_configs.items():
    print(key)
    for stimulus in value['stimuli']:
        print('\t', stimulus)

Step1
	 {u'delay': 700, u'amp': 0.458, u'duration': 2000, u'totduration': 3000}
	 {u'delay': 0, u'amp': -0.126, u'duration': 3000, u'totduration': 3000}
Step3
	 {u'delay': 700, u'amp': 0.95, u'duration': 2000, u'totduration': 3000}
	 {u'delay': 0, u'amp': -0.126, u'duration': 3000, u'totduration': 3000}
Step2
	 {u'delay': 700, u'amp': 0.562, u'duration': 2000, u'totduration': 3000}
	 {u'delay': 0, u'amp': -0.126, u'duration': 3000, u'totduration': 3000}
bAP
	 {u'delay': 295, u'amp': 1.9, u'duration': 5, u'totduration': 600}


And they can be automatically loaded

In [26]:
import l5pc_evaluator
fitness_protocols = l5pc_evaluator.define_protocols()
print('\n'.join('%s' % protocol for protocol in fitness_protocols.values()))

bAP:
  stimuli:
    Square pulse amp 1.900000 delay 295.000000 duration 5.000000 totdur 600.000000 at somatic[0](0.5)
  recordings:
    bAP.soma.v: v at somatic[0](0.5)
    bAP.dend1.v: v at 660.000000 micron from soma in apical
    bAP.dend2.v: v at 800.000000 micron from soma in apical

Step3:
  stimuli:
    Square pulse amp 0.950000 delay 700.000000 duration 2000.000000 totdur 3000.000000 at somatic[0](0.5)
    Square pulse amp -0.126000 delay 0.000000 duration 3000.000000 totdur 3000.000000 at somatic[0](0.5)
  recordings:
    Step3.soma.v: v at somatic[0](0.5)

Step2:
  stimuli:
    Square pulse amp 0.562000 delay 700.000000 duration 2000.000000 totdur 3000.000000 at somatic[0](0.5)
    Square pulse amp -0.126000 delay 0.000000 duration 3000.000000 totdur 3000.000000 at somatic[0](0.5)
  recordings:
    Step2.soma.v: v at somatic[0](0.5)

Step1:
  stimuli:
    Square pulse amp 0.458000 delay 700.000000 duration 2000.000000 totdur 3000.000000 at somatic[0](0.5)
    Square pulse amp

## eFeatures

For every protocol we need to define which eFeatures will be used as objectives of the optimisation algorithm.

In [27]:
feature_configs = json.load(open('config/features.json'))
pp.pprint(feature_configs)

{ u'Step1': { u'soma': { u'AHP_depth_abs': [-60.3636, 2.3018],
                         u'AHP_depth_abs_slow': [-61.1513, 2.3385],
                         u'AHP_slow_time': [0.1599, 0.0483],
                         u'AP_height': [25.0141, 3.1463],
                         u'AP_width': [3.5312, 0.8592],
                         u'ISI_CV': [0.109, 0.1217],
                         u'adaptation_index2': [0.0047, 0.0514],
                         u'doublet_ISI': [62.75, 9.6667],
                         u'mean_frequency': [6, 1.2222],
                         u'time_to_first_spike': [27.25, 5.7222]}},
  u'Step2': { u'soma': { u'AHP_depth_abs': [-59.9055, 1.8329],
                         u'AHP_depth_abs_slow': [-60.2471, 1.8972],
                         u'AHP_slow_time': [0.1676, 0.0339],
                         u'AP_height': [27.1003, 3.1463],
                         u'AP_width': [2.7917, 0.7499],
                         u'ISI_CV': [0.0674, 0.075],
                         u'adaptat

In [28]:
fitness_calculator = l5pc_evaluator.define_fitness_calculator(fitness_protocols)
print(fitness_calculator)

objectives:
  ( AP_height for {'': u'Step1.soma.v'} with stim start 700 and end 2700, exp mean 25.0141 and std 3.1463 and AP threshold override -20 )
  ( AHP_slow_time for {'': u'Step1.soma.v'} with stim start 700 and end 2700, exp mean 0.1599 and std 0.0483 and AP threshold override -20 )
  ( ISI_CV for {'': u'Step1.soma.v'} with stim start 700 and end 2700, exp mean 0.109 and std 0.1217 and AP threshold override -20 )
  ( doublet_ISI for {'': u'Step1.soma.v'} with stim start 700 and end 2700, exp mean 62.75 and std 9.6667 and AP threshold override -20 )
  ( AHP_depth_abs_slow for {'': u'Step1.soma.v'} with stim start 700 and end 2700, exp mean -61.1513 and std 2.3385 and AP threshold override -20 )
  ( AP_width for {'': u'Step1.soma.v'} with stim start 700 and end 2700, exp mean 3.5312 and std 0.8592 and AP threshold override -20 )
  ( time_to_first_spike for {'': u'Step1.soma.v'} with stim start 700 and end 2700, exp mean 27.25 and std 5.7222 and AP threshold override -20 )
  ( AHP_

## Simulator

We need to define which simulator we will use. In this case it will be Neuron, i.e. the NrnSimulator class

In [29]:
sim = ephys.simulators.NrnSimulator()

## Evaluator

With all the components defined above we can build a cell evaluator

In [30]:
evaluator = ephys.evaluators.CellEvaluator(                                          
        cell_model=l5pc_cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)  

This evaluator can be used to run the protocols. The original parameter values for the Markram et al. 2015 L5PC model are:

In [31]:
release_params = {
    'gamma_CaDynamics_E2.somatic': 0.000609,
    'gSKv3_1bar_SKv3_1.somatic': 0.303472,
    'gSK_E2bar_SK_E2.somatic': 0.008407,
    'gCa_HVAbar_Ca_HVA.somatic': 0.000994,
    'gNaTs2_tbar_NaTs2_t.somatic': 0.983955,
    'decay_CaDynamics_E2.somatic': 210.485284,
    'gCa_LVAstbar_Ca_LVAst.somatic': 0.000333
}


Running the responses is as easy as passing the protocols and parameters to the evaluator. (The line below will take some time to execute)

In [32]:
release_responses = evaluator.run_protocols(protocols=fitness_protocols.values(), param_values=release_params)

Exception: Traceback (most recent call last):
  File "/home/hbpschool2016/local/pythonenv/local/lib/python2.7/site-packages/bluepyopt/ephys/protocols.py", line 127, in _run_func
    cell_model.instantiate(sim=sim)
  File "/home/hbpschool2016/local/pythonenv/local/lib/python2.7/site-packages/bluepyopt/ephys/models.py", line 222, in instantiate
    param.instantiate(sim=sim, icell=self.icell)
  File "/home/hbpschool2016/local/pythonenv/local/lib/python2.7/site-packages/bluepyopt/ephys/parameters.py", line 300, in instantiate
    self.value_scale_func(self.value, seg, sim=sim))
AttributeError: 'nrn.Segment' object has no attribute 'gNaTs2_tbar_NaTs2_t'


We can now plot all the responses

In [ ]:
def plot_responses(responses):
    fig, axes = plt.subplots(len(responses), figsize=(10,10))
    for index, (resp_name, response) in enumerate(sorted(responses.items())):
        axes[index].plot(response['time'], response['voltage'], label=resp_name)
        axes[index].set_title(resp_name)
    fig.tight_layout()
    fig.show()
plot_responses(release_responses)

Running an optimisation of the parameters now has become very easy. 
Of course running the L5PC optimisation will require quite some computing resources. 

To show a proof-of-concept, we will only run 2 generations, with 2 offspring individuals per generations.
If you want to run all full optimisation, you should run for 100 generations with an offspring size of 100 individuals. 

In [ ]:
opt = bpopt.optimisations.DEAPOptimisation(evaluator=evaluator, offspring_size=10)

final_pop, halloffame, log, hist = opt.run(max_ngen=10, cp_filename='checkpoints/checkpoint.pkl')

'''
import pickle

cp = pickle.load(open('checkpoints/checkpoint.pkl'))

final_pop = cp['generation']
hist = cp['history']
halloffame = cp['halloffame']
log = cp['logbook']
'''
#create pickle file

The first individual in the hall of fame will contain the best solution found.

In [ ]:
print(halloffame[0])

These are the raw parameter values. 
The evaluator object can convert this in a dictionary, so that we can see the parameter names corresponding to these values.

In [ ]:
best_params = evaluator.param_dict(halloffame[0]) #take the best parameters
print(pp.pprint(best_params))

Then we can run the fitness protocols on the model with these parameter values

In [ ]:
best_responses = evaluator.run_protocols(protocols=fitness_protocols.values(), param_values=best_params)

And then we can also plot these responses. 

When you ran the above optimisation with only 2 individuals and 2 generations, this 'best' model will of course be very low quality.

In [ ]:
plot_responses(best_responses)

Let's visualize the objectives of the optimisation. First we need to create a dictionary of the responses
of the best individual, that is the NEURON simulation of the voltage trace for each protocol with the parameters that correspond to the best individual. Remember that an individual is just a set of our model's parameters.

In [ ]:
fitness_protocols = evaluator.fitness_protocols

responses = {}
nrn = ephys.simulators.NrnSimulator()
for protocol in fitness_protocols.values():
    response = protocol.run(
                            cell_model=opt.evaluator.cell_model,
                            param_values=best_params,
                            sim=nrn
                            )
    responses.update(response)

responses
#create dictionary with the responses

Plot the objectives of the responses, that is the deviation (in standard deviations) of the features from the values in the input features.json file.

In [ ]:
objectives = opt.evaluator.fitness_calculator.calculate_scores(responses)

f, ax = plt.subplots()

ytick_pos = [x + 0.5 for x in range(len(objectives.keys()))]
ax.barh(ytick_pos,
              objectives.values(),
              height=0.5,
              align='center',
              color='#779ECB')

ax.set_yticks(ytick_pos) 
ax.set_yticklabels(objectives.keys(), size='x-small') 
ax.set_ylim(-0.5, len(objectives.values()) + 0.5) 
ax.set_xlabel('Objective value (# std)') 
ax.set_ylabel('Objectives') 

plt.tight_layout()

Let's now plot the traces for all the individuals in the hall of fame. In other words, for each parameter set = individual let's plot the respective trace.

First we create a list for all the individuals, in a simimilar manner as above, but for all the individuals in the hall of fame this time.

In [ ]:
hof_responses = []
for best_ind in halloffame:
    best_ind_dict = evaluator.param_dict(best_ind)
    hof_responses.append(evaluator.run_protocols(evaluator.fitness_protocols.values(),
                                             param_values=best_ind_dict))
    
#iterate on all the halloffame

In [ ]:
traces = sorted(hof_responses[0].keys()) #names of the traces

traces

Then we plot the traces with the best one in deep blue

In [ ]:
import numpy as np

def plot_multiple_responses(responses, fig): #plot the responses
    '''creates 6 subplots for step{1,2,3} and dAP traces, and plots all the responses on them'''

    plot_count = len(traces)
    
    # one subplot for each trace
    ax = [fig.add_subplot(plot_count, 1, i + 1) for i in range(plot_count)]

    # to color the last one deep blue
    overlay_count = len(responses)
    
    for index, trace in enumerate(traces):  
        for n, response in enumerate(reversed(responses)): # worst to best individual
            
            color='lightblue' if n < overlay_count - 1 else 'blue'
            
            ax[index].plot(response[trace]['time'],
                           response[trace]['voltage'],
                           color=color,
                           linewidth=1)

        # formatting of the subplots
        ax[index].set_xlabel('Time (ms)')
        ax[index].set_ylabel('Voltage (mV)')
        ax[index].text(0.01, 0.7, trace, transform=ax[index].transAxes, fontsize=10,
        verticalalignment='top')
        #ax[index].set_xlim(80, 200)
            

        
f = plt.figure(figsize=(10,10))
plot_multiple_responses(hof_responses, f)
#Light blue lines : not spiking at all

In [ ]:
responses

And finally the fitness function

In [ ]:
import numpy

logs=log
gen_numbers = log.select('gen')
min_fitness = numpy.array(log.select('min'))
max_fitness = log.select('max')
mean_fitness = numpy.array(log.select('avg'))
std_fitness = numpy.array(log.select('std'))

fig, ax = plt.subplots(1, figsize=(10, 5), facecolor='white')

std = std_fitness
mean = mean_fitness
minimum = min_fitness
stdminus = mean - std                                                           
stdplus = mean + std

ax.plot(                                                                      
    gen_numbers,                                                                
    mean,                                                                       
    color='black',                                                              
    linewidth=2,                                                                
    label='population average')                                                 

ax.fill_between(                                                              
    gen_numbers,                                                                
    stdminus,                                                                   
    stdplus,                                                                    
    color='lightgray',                                                          
    linewidth=2,                                                                
    label=r'population standard deviation')                                     

ax.plot(                                                                      
    gen_numbers,                                                                
    minimum,                                                                    
    color='red',                                                                
    linewidth=2,                                                                
    label='population minimum')                                                 

ax.set_xlim(min(gen_numbers) - 1, max(gen_numbers) + 1)                       
ax.set_xlabel('Generation #')                                                 
ax.set_ylabel('Sum of objectives')                                            
ax.set_ylim([0, max(stdplus)])                                                
ax.legend()                        


fig.show()
#Mean + standard deviation + red(the best one)